In [12]:
import json

In [13]:
import os

In [14]:
import pandas as pd

In [15]:
result_directories = [
    "baseline_results/baseline_results_10_dims",
    "baseline_results/baseline_results_50_dims",
    "baseline_results/baseline_results_100_dims",
    "baseline_results/baseline_results_gurobi"
]

In [16]:
result_directories = [
    "../baseline_results_nn/",
]

In [31]:
def extract_nn_part(file_name):
    nn_fname = file_name.strip(".json").split("-")[-1]
    return nn_fname

In [56]:
def extract_dim_from_nn_fname(nn_fname):
    dimension = int(nn_fname.split("_")[-2])
    return dimension

In [57]:
def extract_hidden_size_from_nn_fname(nn_fname):
    hidden_size = int(nn_fname.split("_")[-1])
    return hidden_size

In [32]:
def extract_solver(file_name):
    candidates = ["scipy", "gurobi"]
    for cand in candidates:
        if cand in file_name:
            return cand
    
    assert False, "Solver is not recognized from candidates {} for file {}.".format(candidates, file_name)

In [33]:
def extract_algorithm(file_name):
    candidates = ["direct", "shgo", "differential_evolution", "dual_annealing", "brute", "basinhopping"]
    for cand in candidates:
        if cand in file_name:
            return cand
    
    assert False, "Algorithm is not recognized from candidates {} for file {}.".format(candidates, file_name)

In [34]:
def extract_seed(file_name):
    seed = file_name.strip(".json").split("-")[1].split("_")[-1]
    assert str.isdigit(seed), "Seed {} is not an integer.".format(seed)
    return int(seed)

In [35]:
"NeuralNetworkOneLayer_-1_scipy_direct_0-nn_one_layer_ackley_2_16.json".split("-.")

['NeuralNetworkOneLayer_-1_scipy_direct_0-nn_one_layer_ackley_2_16.json']

In [63]:
results = []

for result_dir in result_directories:
    path = os.path.join(result_dir, "results")
    for fn in os.listdir(path):
#         print(fn)
        entry_dict = {}
#         function = extract_function(fn)
        solver = extract_solver(fn)
        nn_fname = extract_nn_part(fn)
        if solver == "gurobi":
            algorithm = None
        else:
            algorithm = extract_algorithm(fn)
        dimension = extract_dim_from_nn_fname(nn_fname)
        hidden_size = extract_hidden_size_from_nn_fname(nn_fname)
        seed = extract_seed(fn)
        
        info_dict = {
#             "function": function,
            "solver": solver,
            "algorithm": algorithm,
            "dimension": dimension,
            "hidden_size": hidden_size,
            "nn_fname": nn_fname,
            
            "seed": seed,
        }
        with open(os.path.join(path, fn)) as f:
            result_dict = json.load(f)
        entry_dict.update(info_dict)
        entry_dict.update(result_dict)
        
        results.append(entry_dict)

In [64]:
for result in results:
    if result['algorithm'] == None:
        result['algorithm'] = 'gurobi'

In [65]:
df = pd.DataFrame(results)
df['timeout_reached'] = df['timeout_reached'].fillna(False)

In [66]:
result_df = df.sort_values(by=['dimension', 'hidden_size', 'nn_fname', 'solver', 'algorithm', 'seed'])

In [67]:
import pandas
pandas.set_option('display.max_rows', 180)

result_df

,solver,algorithm,dimension,hidden_size,nn_fname,seed,x,obj_val,time_elapsed,timeout_reached
29,scipy,basinhopping,2,16,nn_one_layer_ackley_2_16,0,"[3.8921295408305743, 11.581883957817544]",18.515703,2.505791,False
36,scipy,basinhopping,2,16,nn_one_layer_ackley_2_16,1,"[-3.80470615411847, 9.129294999979951]",15.886199,2.789560,False
6,scipy,basinhopping,2,16,nn_one_layer_ackley_2_16,2,"[-2.9515114335607686, -30.509983781736977]",22.100773,2.842209,False
154,scipy,basinhopping,2,16,nn_one_layer_ackley_2_16,3,"[0.6111341360353965, 8.185782187358598]",14.872093,2.362648,False
109,scipy,basinhopping,2,16,nn_one_layer_ackley_2_16,4,"[29.1431824512606, 7.0182810272130824]",21.675371,2.586025,False
70,scipy,brute,2,16,nn_one_layer_ackley_2_16,0,"[0.6543061931107113, -0.07545062987912676]",8.816602,2.785541,False
67,scipy,brute,2,16,nn_one_layer_ackley_2_16,1,"[0.6543061931107113, -0.07545062987912676]",8.816602,2.723691,False
124,scipy,brute,2,16,nn_one_layer_ackley_2_16,2,"[0.6543061931107113, -0.07545062987912676]",8.816602,2.637029,False
146,scipy,brute,2,16,nn_one_layer_ackley_2_16,3,"[0.6543061931107113, -0.07545062987912676]",8.816602,2.637624,False
46,scipy,brute,2,16,nn_one_layer_ackley_2_16,4,"[0.6543061931107113, -0.07545062987912676]",8.816602,2.667028,False


In [68]:
statistics_df = df.groupby(by=['dimension', 'hidden_size', 'nn_fname', 'solver', 'algorithm']).agg(
    obj_mean=('obj_val', 'mean'),
    obj_min=('obj_val', 'min'),
    obj_max=('obj_val', 'max'),
    time_mean=('time_elapsed', 'mean'),
    time_min=('time_elapsed', 'min'),
    time_max=('time_elapsed', 'max'),
    num_seeds=('seed', 'count'),
    num_timeout=('timeout_reached', 'sum'),
).reset_index()

In [69]:
statistics_df

,dimension,hidden_size,nn_fname,solver,algorithm,obj_mean,obj_min,obj_max,time_mean,time_min,time_max,num_seeds,num_timeout
0,2,16,nn_one_layer_ackley_2_16,scipy,basinhopping,18.610028,14.872093,22.100773,2.617247,2.362648,2.842209,5,0
1,2,16,nn_one_layer_ackley_2_16,scipy,brute,8.816602,8.816602,8.816602,2.690183,2.637029,2.785541,5,0
2,2,16,nn_one_layer_ackley_2_16,scipy,differential_evolution,8.820141,8.818261,8.821816,2.757164,2.618578,2.884644,5,0
3,2,16,nn_one_layer_ackley_2_16,scipy,direct,8.816657,8.816657,8.816657,5.665247,5.461331,5.771033,5,0
4,2,16,nn_one_layer_ackley_2_16,scipy,dual_annealing,8.816619,8.816605,8.816646,13.039796,11.852799,13.561692,5,0
5,2,16,nn_one_layer_ackley_2_16,scipy,shgo,9.052660,9.052660,9.052660,2.185509,2.088339,2.231488,5,0
6,2,16,nn_one_layer_levy_2_16,scipy,basinhopping,0.074160,-0.551064,2.357451,2.612831,2.535311,2.794178,5,0
7,2,16,nn_one_layer_levy_2_16,scipy,brute,-0.551064,-0.551064,-0.551064,2.592724,2.535957,2.706497,5,0
8,2,16,nn_one_layer_levy_2_16,scipy,differential_evolution,-0.551064,-0.551064,-0.551064,2.621584,2.506058,2.760783,5,0
9,2,16,nn_one_layer_levy_2_16,scipy,direct,-0.551064,-0.551064,-0.551064,5.389431,5.310930,5.483335,5,0


In [18]:
result_df.to_csv("result.csv", index=False)
statistics_df.to_csv("stats.csv", index=False)

In [70]:
result_df.to_csv("result_nn.csv", index=False)
statistics_df.to_csv("stats_nn.csv", index=False)